In [ ]:
from kafka import KafkaConsumer
import time
import json
import queue
#Para vaciar la salida de una celda del notebook
from IPython.display import clear_output 

## Definición del consumer para el servicio que está en cloud [Aiven](https://aiven.io/)
### Este servicio requiere que estén en la carpeta ./cert estén los ficheros con los certificados y el key de la aplicación


In [ ]:
"""
Este consumer utiliza un servicio de Kafka en la nube 

Requiere que en la carpeta ./cert estén los ficheros 
con los certificados y el key de la aplicación
"""

consumerCloud = KafkaConsumer(
    "accidentes",
    bootstrap_servers="kafka-194e1e33-jose-30ef.aivencloud.com:26426",
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    client_id="demo-client",
    group_id="demo-group",
    security_protocol="SSL",
    ssl_cafile="./cert/ca.pem",
    ssl_certfile="./cert/service.cert",
    ssl_keyfile="./cert/service.key"
)



## Se presentan en un panel los últimos 10 mensajes recibidos por kafka

In [ ]:
"""
Presenta en pantalla los últimso 10 accidentes que se han producido
para que sean monitorizados

En este servicio de kafka el topic 'accidentes' está grabado como un json,
por eso se utiliza el deserializador
"""
NUM_ALERTS = 10
alerts=[None for i in range(NUM_ALERTS)] #inicializar las alertas
current = 0
while True:
    raw_msgs = consumerCloud.poll(timeout_ms=1000, max_records=1)
    for tp, msgs in raw_msgs.items():
        for msg in msgs:
            if alerts [NUM_ALERTS-1] == None or alerts[NUM_ALERTS-1]['acc'] != msg.value['acc']:
                #Elimina la alerta más antigua
                alerts.pop(0)
                #incluye el nuevo elemento al final de la lista
                alerts.append(msg.value)
        #Limpia la salida de la celda para evitar el scroll
        clear_output(wait=True)
        #Los elementos más nuevos están al final de la lista
        print ('Panel -----------------------------------------------------')
        for i in range(NUM_ALERTS-1,0,-1):
            if alerts[i] != None:
                print (alerts[i]['acc'],'->\t',alerts[i]['calle'],alerts[i]['num'])
        print ('-----------------------------------------------------------')
    time.sleep(1)


## Definición del consumer para el servicio que está en uno de los equipos del aula

In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    "accidentes",
    bootstrap_servers=["10.0.7.109:9092"],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='cluster2',
    value_deserializer=lambda x: x.decode('utf-8'))


## Se presentan en un panel los últimos 10 mensajes recibidos por kafka

In [12]:
"""
Presenta en pantalla los últimso 10 accidentes que se han producido
para que sean monitorizados

En este servicio de kafka el topic 'accidentes' está grabado como un json,
por eso se utiliza el deserializador
"""
NUM_ALERTS = 10
alerts=[None for i in range(NUM_ALERTS)] #inicializar las alertas
current = 0
while True:
    raw_msgs = consumer.poll(timeout_ms=1000, max_records=1)
    #print ('Poll ralizado, mensajes {}'.format(len(raw_msgs.items())))
    for tp, msgs in raw_msgs.items():
        #Limpia la salida de la celda para evitar el scroll
        #print ('cargar {} mensajes'.format(len(msgs)))
        for msg in msgs:
            data = msg.value.split(',')
            acc = data[6]
            calle = data[4].strip()
            num = data[5]
            if alerts [NUM_ALERTS-1] == None or alerts[NUM_ALERTS-1]['acc'] != acc:
                #Elimina la alerta más antigua
                alerts.pop(0)
                #incluye en la cabecera
                alerts.append({'acc':acc, 'calle':calle, 'num':num})
        clear_output(wait=True)
        #Los elementos más nuevos están al final de la lista
        print ('Panel -----------------------------------------------------')
        for i in range(NUM_ALERTS-1,0,-1):
            if alerts[i] != None:
                print (alerts[i]['acc'],'->\t',alerts[i]['calle'],alerts[i]['num'])
        print ('-----------------------------------------------------------')
    time.sleep(1)


Panel -----------------------------------------------------
2017/1491 ->	 CALLE DE ALCALA NUM 716
2017/1565 ->	 AUTOVIA  M-30 CALZADA 1 - CALLE DE ALCALA 
2017/1489 ->	 AVENIDA DE LOS ANDES - GLORIETA DEL MAR DE CRISTAL 0
2017/1534 ->	 AVENIDA DE MENENDEZ PELAYO - CALLE DEL DOCE DE OCTUBRE 0
2017/1571 ->	 AUTOVIA  A-4 ENTRADA KM. 4000
2017/1549 ->	 PASEO DE LA CASTELLANA NUM 109
2017/1530 ->	 AUTOVIA  M-23 ENTRADA KM. 1000
2017/1546 ->	 CALLE DE GONZALEZ DAVILA - CAMINO DE LAS HORMIGUERAS 0
2017/1524 ->	 PASEO DE LA CASTELLANA NUM 84
-----------------------------------------------------------


KeyboardInterrupt: 